In [2]:
import pandas as pd

In [3]:
csv_path = "Houses Dataset/HousesInfo.txt"
cols = ["bedrooms","bathrooms", "area", "zipcode", "price"]
df = pd.read_csv(csv_path, sep=" ", header=None, names=cols)

In [12]:
import os
import glob
import numpy as np
import cv2

In [15]:
images = []
inputPath = 'Houses Dataset'


for i in df.index.values:
    basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
    housePaths = sorted(list(glob.glob(basePath)))

    inputImages = []
    outputImage = np.zeros((64, 64, 3), dtype="uint8")

    for housePath in housePaths:
        image = cv2.imread(housePath)
        image = cv2.resize(image, (32, 32))
        inputImages.append(image)

    outputImage[0:32, 0:32] = inputImages[0]
    outputImage[0:32, 32:64] = inputImages[1]
    outputImage[32:64, 32:64] = inputImages[2]
    outputImage[32:64, 0:32] = inputImages[3]
    images.append(outputImage)

In [14]:
# return our set of images
images = np.array(images)

In [17]:
images.shape

(535, 64, 64, 3)

In [18]:
images = images / 255.0

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
trainAttrX, testAttrX, trainImagesX, testImagesX = train_test_split(df, images, test_size=0.25, random_state=42)

In [22]:
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

In [31]:
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, BatchNormalization
from keras.layers import MaxPooling2D, Flatten, Dropout

inputShape = (64, 64, 3)
chanDim = -1

# define the model input
inputs = Input(shape=inputShape)
x = inputs

# CONV => RELU => BN => POOL
x = Conv2D(16, (3, 3), padding="same", activation='relu')(x)
x = BatchNormalization(axis=chanDim)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3), padding="same", activation='relu')(x)
x = BatchNormalization(axis=chanDim)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), padding="same", activation='relu')(x)
x = BatchNormalization(axis=chanDim)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# flatten the volume, then FC => RELU => BN => DROPOUT
x = Flatten()(x)
x = Dense(16, activation='relu')(x)
x = BatchNormalization(axis=chanDim)(x)
x = Dropout(0.5)(x)

x = Dense(4, activation='relu')(x)
x = Dense(1, activation="linear")(x)

# construct the CNN
model = Model(inputs, x)

In [32]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_14 (Conv2D)          (None, 64, 64, 16)        448       
                                                                 
 batch_normalization_14 (Ba  (None, 64, 64, 16)        64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 32, 32, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 batch_normalization_15 (Ba  (None, 32, 32, 32)        128   

In [33]:
model.compile(loss="mean_absolute_percentage_error", optimizer='adam')

In [34]:
# train the model
print("[INFO] training model...")
model.fit(trainImagesX, trainY, validation_data=(testImagesX, testY),
          epochs=200, batch_size=8)



[INFO] training model...
Epoch 1/200
51/51 [==============================] - 2s 16ms/step - loss: 766.2432 - val_loss: 767.9109
Epoch 2/200
51/51 [==============================] - 1s 13ms/step - loss: 546.1312 - val_loss: 1921.3055
Epoch 3/200
51/51 [==============================] - 1s 14ms/step - loss: 363.1629 - val_loss: 2447.9749
Epoch 4/200
51/51 [==============================] - 1s 13ms/step - loss: 299.2203 - val_loss: 2328.7134
Epoch 5/200
51/51 [==============================] - 1s 13ms/step - loss: 224.3407 - val_loss: 2573.8315
Epoch 6/200
51/51 [==============================] - 1s 13ms/step - loss: 199.5705 - val_loss: 2059.5183
Epoch 7/200
51/51 [==============================] - 1s 13ms/step - loss: 173.9633 - val_loss: 978.8597
Epoch 8/200
51/51 [==============================] - 1s 13ms/step - loss: 130.9521 - val_loss: 670.8191
Epoch 9/200
51/51 [==============================] - 1s 13ms/step - loss: 127.9842 - val_loss: 361.9952
Epoch 10/200
51/51 [==============

51/51 [==============================] - 1s 13ms/step - loss: 75.7546 - val_loss: 102.4091
Epoch 81/200
51/51 [==============================] - 1s 14ms/step - loss: 88.1583 - val_loss: 81.9052
Epoch 82/200
51/51 [==============================] - 1s 13ms/step - loss: 61.6267 - val_loss: 149.4530
Epoch 83/200
51/51 [==============================] - 1s 13ms/step - loss: 53.0198 - val_loss: 79.2617
Epoch 84/200
51/51 [==============================] - 1s 14ms/step - loss: 53.4648 - val_loss: 66.5255
Epoch 85/200
51/51 [==============================] - 1s 13ms/step - loss: 50.4785 - val_loss: 68.3201
Epoch 86/200
51/51 [==============================] - 1s 13ms/step - loss: 57.8108 - val_loss: 68.2759
Epoch 87/200
51/51 [==============================] - 1s 14ms/step - loss: 58.8331 - val_loss: 74.6851
Epoch 88/200
51/51 [==============================] - 1s 13ms/step - loss: 54.8203 - val_loss: 76.6159
Epoch 89/200
51/51 [==============================] - 1s 13ms/step - loss: 50.2695 -

51/51 [==============================] - 1s 13ms/step - loss: 48.8466 - val_loss: 64.5826
Epoch 160/200
51/51 [==============================] - 1s 13ms/step - loss: 42.2799 - val_loss: 65.0256
Epoch 161/200
51/51 [==============================] - 1s 13ms/step - loss: 45.8097 - val_loss: 70.3107
Epoch 162/200
51/51 [==============================] - 1s 13ms/step - loss: 43.4357 - val_loss: 60.2731
Epoch 163/200
51/51 [==============================] - 1s 13ms/step - loss: 46.6683 - val_loss: 57.9689
Epoch 164/200
51/51 [==============================] - 1s 13ms/step - loss: 41.9909 - val_loss: 63.4345
Epoch 165/200
51/51 [==============================] - 1s 13ms/step - loss: 40.3636 - val_loss: 66.3229
Epoch 166/200
51/51 [==============================] - 1s 13ms/step - loss: 42.3913 - val_loss: 62.7722
Epoch 167/200
51/51 [==============================] - 1s 13ms/step - loss: 45.6730 - val_loss: 59.7778
Epoch 168/200
51/51 [==============================] - 1s 13ms/step - loss: 41

In [35]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict(testImagesX)

[INFO] predicting house prices...
5/5 [==============================] - 0s 8ms/step


In [37]:
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

In [41]:
import locale

mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] avg. house price: $589,362.81, std house price: $509,026.07
[INFO] mean: 62.15%, std: 80.87%
